In [1]:
import pandas as pd
import numpy as np

In [2]:
pwd

u'/Users/neeraj/Dropbox/Twitter2/datajam/scripts'

## Read the data

In [3]:
shmads = pd.read_csv('../data/Sept12Oct22_v2.csv')

In [7]:
dat = pd.read_csv(open('../data/%s.txt' % "coords", 'rU'),
                            sep='\t',
                            encoding='utf-8',
                            engine='c',
                            error_bad_lines=False)

Skipping line 180: expected 6 fields, saw 8
Skipping line 412: expected 6 fields, saw 8
Skipping line 1783: expected 6 fields, saw 8
Skipping line 2022: expected 6 fields, saw 8
Skipping line 2103: expected 6 fields, saw 8
Skipping line 2148: expected 6 fields, saw 8
Skipping line 2216: expected 6 fields, saw 8
Skipping line 3201: expected 6 fields, saw 8
Skipping line 3225: expected 6 fields, saw 8
Skipping line 3479: expected 6 fields, saw 8
Skipping line 4228: expected 6 fields, saw 8
Skipping line 4797: expected 6 fields, saw 8
Skipping line 4937: expected 6 fields, saw 8
Skipping line 5254: expected 6 fields, saw 8
Skipping line 5350: expected 6 fields, saw 8
Skipping line 5402: expected 6 fields, saw 8
Skipping line 5722: expected 6 fields, saw 8
Skipping line 6063: expected 6 fields, saw 8
Skipping line 6570: expected 6 fields, saw 8
Skipping line 6661: expected 6 fields, saw 8
Skipping line 6778: expected 6 fields, saw 8
Skipping line 6927: expected 6 fields, saw 8
Skipping lin

In [34]:
dat.shape

(194571, 6)

In [8]:
dat.head()

,Tag,Date,Tweet,Lat,Lon,n_Followers
0,ALCOHOLIC,Sun Oct 12 05:31:21 +0000 2014,@Influenster: You've got to try these alcoholi...,41.672442,-81.273371,138
1,SUICIDE,Sun Oct 12 05:41:18 +0000 2014,Check It Fest: the youth suicide awareness fes...,-27.965333,153.416862,1106
2,SUICIDE,Sun Oct 12 05:47:59 +0000 2014,#nowplaying Suicide Silence - Witness The Addi...,34.967096,135.772691,45
3,DEPRESSED,Sun Oct 12 06:53:27 +0000 2014,Haven't spoken to baby and I'm getting depressed,41.754551,-72.710976,147
4,DEPRESSED,Sun Oct 12 07:09:17 +0000 2014,"I was all happy this whole day, now I'm all ba...",33.144335,-110.124977,573


In [9]:
shmads = shmads.dropna(subset=['Neg'])

In [10]:
shmads.shape

(1318, 22)

In [11]:
T = np.array(shmads.Tweets)

## Ascii assertion

In [12]:
Tx = [x.decode().encode('ascii') for x in T]

In [13]:
Tar = np.array(shmads.Neg)

## Get wordcounts

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
count_vect = CountVectorizer()

In [16]:
X_train_counts = count_vect.fit_transform(Tx)

In [17]:
X_train_counts.shape

(1318, 5292)

In [18]:
X_train_counts

<1318x5292 sparse matrix of type '<type 'numpy.int64'>'
	with 19313 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

In [20]:
X_train_tf.shape

(1318, 5292)

In [21]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(1318, 5292)

In [22]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, Tx)

In [23]:
X_new_counts = count_vect.transform(Tx)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

In [24]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])
 

In [25]:
text_clf = text_clf.fit(Tx[:800], Tar[:800])

In [26]:
predicted = text_clf.predict(Tx[501:1000])

In [27]:
shmads.Neg[60:70]

60    0
61    0
62    0
63    0
64    1
65    1
66    1
67    1
68    1
69    1
Name: Neg, dtype: float64

In [28]:
from sklearn.grid_search import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 3)],
             'tfidf__use_idf': (True, False),
             'clf__alpha': (1e-2, 1e-3)}

In [29]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [30]:
gs_clf = gs_clf.fit(Tx[1:1317], Tar[1:1317])

## Apply the classifier to new data

In [32]:
def rate_tweets(term):
    term_data = pd.read_csv(open('../data/%s.txt' % term, 'rU'),
                            sep='\t',
                            encoding='utf-8',
                            engine='c',
                            error_bad_lines=False)
    term_Tx = [str(x).decode().encode('ascii') for x in term_data.Tweet]
    predict = gs_clf.predict(term_Tx)
    term_data["Neg"] = predict
    term_data.to_json('../data/all-data-with-%s.json' % term, force_ascii=False, orient = 'records')
    term_data.to_csv('../data/all-data-with-%s.tsv' % term, encoding='utf-8', index=False, sep='\t')

## Writing out the data

In [33]:
rate_tweets('coords')

Skipping line 180: expected 6 fields, saw 8
Skipping line 412: expected 6 fields, saw 8
Skipping line 1783: expected 6 fields, saw 8
Skipping line 2022: expected 6 fields, saw 8
Skipping line 2103: expected 6 fields, saw 8
Skipping line 2148: expected 6 fields, saw 8
Skipping line 2216: expected 6 fields, saw 8
Skipping line 3201: expected 6 fields, saw 8
Skipping line 3225: expected 6 fields, saw 8
Skipping line 3479: expected 6 fields, saw 8
Skipping line 4228: expected 6 fields, saw 8
Skipping line 4797: expected 6 fields, saw 8
Skipping line 4937: expected 6 fields, saw 8
Skipping line 5254: expected 6 fields, saw 8
Skipping line 5350: expected 6 fields, saw 8
Skipping line 5402: expected 6 fields, saw 8
Skipping line 5722: expected 6 fields, saw 8
Skipping line 6063: expected 6 fields, saw 8
Skipping line 6570: expected 6 fields, saw 8
Skipping line 6661: expected 6 fields, saw 8
Skipping line 6778: expected 6 fields, saw 8
Skipping line 6927: expected 6 fields, saw 8
Skipping lin